In [1]:
import fitz  # PyMuPDF for PDF handling
import pandas as pd  # For DataFrame handling
import re  # Regular Expressions for text processing

In [10]:
def extract_text_from_paragraph(page, bbox):
    """Extract text from a specific bounding box on a page."""
    text = page.get_text("text", clip=bbox)
    return text.strip()

In [11]:
def extract_images_from_paragraph(page, bbox):
    """Extract images from a specific bounding box on a page."""
    # You'll need to implement logic to extract images from the specified bounding box
    images = []  # Placeholder for images
    return images

In [1]:
def extract_headings_by_font(page):
    headings = []
    for block in page.get_text("dict")["blocks"]:
        # Assuming headings have larger font sizes (adjust threshold as needed)
        if block.get("size", 0) > 12:  # Example threshold for font size
            headings.append(block["text"])
    return headings

In [2]:
def extract_headings_by_position(page):
    headings = []
    for block in page.get_text("dict")["blocks"]:
        # Assuming headings are centered (adjust based on document layout)
        if block.get("x0", 0) == block.get("x1", 0) / 2:
            headings.append(block["text"])
    return headings

In [3]:
def extract_headings_from_toc(toc):
    headings = [entry[1] for entry in toc]  # Assuming TOC structure: (page_num, heading)
    return headings

In [4]:
def extract_headings_by_regex(page):
    headings = []
    for block in page.get_text("dict")["blocks"]:
        # Example: Looking for text starting with "Chapter" followed by a number
        if re.match(r"Chapter\s\d+", block["text"]):
            headings.append(block["text"])
    return headings

In [5]:
def extract_headings_by_structure(page):
    # Custom logic based on specific document structure, e.g., looking for <h1>, <h2> tags
    # Implement your logic here
    pass

In [6]:
def extract_headings(page, method="font"):
    if method == "font":
        return extract_headings_by_font(page)
    elif method == "position":
        return extract_headings_by_position(page)
    elif method == "toc":
        return extract_headings_from_toc(page.get_toc())
    elif method == "regex":
        return extract_headings_by_regex(page)
    elif method == "structure":
        return extract_headings_by_structure(page)
    else:
        return []

In [12]:
def parse_pdf(file_path, title_extraction_method="font"):
    doc = fitz.open(file_path)
    pages = len(doc)

    parsed_data = []
    for page_num in range(pages):
        page = doc.load_page(page_num)
        headings = extract_headings(page, method=title_extraction_method)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if b["type"] == 0:  # Text block
                bbox = b["bbox"]
                text = extract_text_from_paragraph(page, bbox)
                images = extract_images_from_paragraph(page, bbox)
                title = headings[page_num] if page_num < len(headings) else "Untitled"

                parsed_content = {
                    "page_numbers": [page_num + 1],
                    "title": title,
                    "text": text,
                    "images": images,
                    "tables": []  # Placeholder for tables
                }
                parsed_data.append(parsed_content)

    doc.close()
    return parsed_data

In [13]:
# Rest of the code remains unchanged

# Usage
file_path = "a.pdf"
parsed_data = parse_pdf(file_path, title_extraction_method="font")
df = pd.DataFrame(parsed_data)
print(df)

NameError: name 'fitz' is not defined